In [ ]:
!pip install --quiet google-cloud-aiplatform
!pip install --quiet google-generativeai
!pip install --quiet openai

In [ ]:
import openai
import pandas as pd
import json
import time


In [ ]:
subreddit_data = [
    {
        "csv_path": "/content/worldnews_heavily_downvoted_comments (1) (2) - worldnews_heavily_downvoted_comments (1) (2).csv.csv",
        "subreddit": "r/worldnews",
        "rules": """
                Disallowed comments
                The moderators have the final say on whether or not a user broke the sub's rules.

                The moderators can ban any user that breaks the sub's rules without providing a warning.

                The moderators can ban any troll without providing a warning. The moderators have the final say on whether or not a user is a troll.

                1 Bigotry / Other offensive content - especially hate speech directed towards an entire group of people who share a race, gender, sexual orientation, gender identity, ethnicity, religion, nationality, social status, immigration status or disability. This also applies to user names. Do not justify bigotry. Do not incite hatred. The moderators have the final say on what is considered offensive enough to remove.

                2 Personal attacks on other users - You may criticize a user's comment but not their person. Such attacks shut down discussion. Accusing another user of being a "shill" is considered a personal attack. Do not attack other users or the mods. Stick to the issues and remain civil. No one else's behavior justify you breaking the rules. You can click the report option located under any comment if you think a user is breaking the sub's rules.

                3 Memes/GIFs - Memes may show up in image or in text form. Both are not allowed.

                4 Unlabeled NSFW images/videos - This refers to newsworthy, relevant NSFW content. Porn or shock images unrelated to the story at hand will always be removed and posting them is an easy way to get banned.

                5 URL Shorteners - If you want your comment to show up right away, don't use link shorteners. They are all removed because they can redirect to malware.

                6 Celebrating death/Advocating violence - Do not celebrate death or wish it upon others. Do not advocate violence. Do not justify terrorism. Do not promote or support legally designated foreign terrorist organizations such as Hamas. Do not promote or support the activities of terrorists such as Hamas, Hezbollah or the Houthis. Do not downplay atrocities. Do not deny atrocities. Do not ask to see pictures or videos of dead bodies or sexual assault--this would include asking people to DM you these items or asking for links to see that type of content.

                7 Genocide denial/Downplaying genocide - Comments denying genocide or downplaying genocide will be removed, and possibly lead to a ban. The moderators have the final say on which comments are removed.

                8 Disinformation/Misinformation - Do not spread disinformation. The moderators have the final say on which comments are removed.

                9 Health disinformation/misinformation - Do not spread falsifiable health information that encourages or poses a significant risk of physical harm to anyone. This includes content that downplays or mocks necessary public health measures to combat the coronavirus pandemic such as taking vaccines or wearing masks. Do not discourage people from taking the COVID vaccine. The moderators have the final say on which comments are removed.
        """
    },
    {
        "csv_path": "/content/legaladvice_heavily_downvoted_finished - legaladvice_heavily_downvoted_comments (1) (2).csv.csv",
        "subreddit": "r/legaladvice",
        "rules": """
                1. Comments should contain a legal answer or a strongly related non-legal answer. If it is not legal advice, do not post. Period. You will be banned.
                2. Personal anecdotes are off-topic.
                3. Explanations of the law in jurisdictions other than the one described in the OP are off-topic.
                4. Opinions on the law or the application of it are off-topic.
                5. Comments serving only to berate others are off-topic.
                6. Expressions of sympathy without corresponding legal help is off-topic.
                7. Be Polite. Disagreements are fine, but disagreement should be done respectfully and with detail. Simply stating "you're wrong" is not appropriate. Similarly, do not respond poorly if challenged respectfully.
                8. Comments should be reasonably detailed and explanatory. "I'm a lawyer so listen to me" isn't an appropriate answer. Credential fights are not appropriate here.
                9. Requests for updates are off-topic.
                10. You will be banned permanently and irrevocably if you use ChatGPT or similar to answer someone's question.
        """
    },
    {
        "csv_path": "/content/askscience_heavily_downvoted_comments_finished - askscience_heavily_downvoted_comments (1).csv.csv",
        "subreddit": "r/askscience",
        "rules": """
            • Top level comments (direct replies to a question, not a reply to another user's comment) must be an answer to the question posed, or a follow-up question to the post.

            • No medical advice, speculation, or personal anecdotes. See more here.

            • Answers should be supported by reputable sources and scientific research. Do not cite yourself as a source.

            • Comments must be civil and on topic. No memes, jokes, or comments consisting solely of a link.

            • No abusive, harassing, offensive or spam comments.

            • Bot and AI tool generated content is not permitted.
        """
    },
    {
        "csv_path": "/content/IAmA_heavily_downvoted_comments (4) (2) - IAmA_heavily_downvoted_comments (4) (2).csv.csv",
        "subreddit": "r/IAmA",
        "rules": """
            Comments will be removed under a few circumstances:

            Abusive or harassing comments

            Comments responding to verification that are unrelated to verification.

            Requests for personal favors from the OP (For example, "OP, can you send me a signed autograph").

            In AMA posts, top-level comments must ask a question. We will remove comments such as "OMG I love you..." and "No questions, just thanks!"

            Comments where there would be no possibility of a real answer, especially where it is deliberately creepy or offensive.

            "I bet OP won't answer this"-type responses, which usually come after the OP has finished responding to questions.

            "Fluff," non-contributing responses from users, responding to all of the OP's comments for karma/attention.

            Repeatedly asking the same question, which violates Reddit's site-wide rules.

            Users attempting to bypass the rules by adding a ? to a nonquestion will be permanently banned from the subreddit.

            Questions must be directed toward the individual(s) doing the IAMA.

            A subreddit or other website organizing and voting for a group comment/question is considered to be vote cheating and is subject to removal. It is a violation of the rules of reddit and risks a sitewide ban.

            Under our policies, astroturfing is the practice of an individual or group of individuals who plant questions in an IAmA post for a particular purpose. This kind of behavior will result in a permanent ban from /r/IAmA.

        """
    },
    {
        "csv_path": "/content/NBA_heavily_downvoted_comments_finished - NBA_heavily_downvoted_comments (1) (2).csv.csv",
        "subreddit": "r/NBA",
        "rules": """
            Prohibited Comments
            1b. Threats, suggesting/wishing harm, or personal insults

            Any comments or posts including threats, suggestions of harm, or personal insults will get you banned, including any threats or wishing harm towards players.

            2b. Racist, sexist, homophobic, or otherwise discriminatory language

            This is an automatic ban. This includes jokes or quotes that include such language. We want everyone, no matter their age, gender, sexuality, or race, to feel welcome in this subreddit.

            3b. Flame bait

            Trash talk is okay, but once it gets too excessive or personal, it's not okay. Do not comment with the sole purpose of trolling or inciting reactions from other users. Flamebaiting includes concern trolling and whataboutism. This will lead to a ban.

            4b. Harmful Misinformation

            Comments or posts that contain harmful misinformation will be removed at moderator discretion. Examples include but are not limited to: COVID misinformation, Political Misinformation, etc.

            5b. Comments/Posts with another user/person’s personal information

            This one is self explanatory, but making comments or posts with another person’s personal information(name, twitter, facebook, phone number, etc) is against Reddit’s site-wide policies and definitely against ours as well.

            6b. Comment Full Edits

            Editing a popular comment or post in order to troll/advertise/make a joke/etc. can result in a ban. Additionally, if you have a highly downvoted comment and you edit the comment to troll, you face a ban.
        """
    }
]

In [ ]:
def analyze_comments_batch(comments, rules):
    comments_str = "\n\n".join([f"{i + 1}: {comment.replace('{', '{{').replace('}', '}}')}" for i, comment in enumerate(comments)])

    messages = [
        {"role": "system", "content": "You are a content moderator analyzing comments for rule violations."},
        {"role": "user", "content": f"""
        Analyze these comments:

        {comments_str}

        Using these subreddit rules:
        {rules}

        Return a JSON object where each entry is keyed by the comment number and follows this exact format:
        {{
            "comment_number": {{
                "unfairly_downvoted": true means doesn't break rules or false meaning breaks rule in subreddit,
                "rule_violated": "only if rule broken, then specify rule title here broken. Otherwise, specify 'null'"
            }},
            ...
        }}
        You must return only valid JSON with no additional text."""}
    ]

    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="hf:Qwen/Qwen2.5-72B-Instruct",
                messages=messages
            )
            response = completion.choices[0].message.content

            response = response.strip()
            if response.startswith('```json'):
                response = response[7:]
            if response.endswith('```'):
                response = response[:-3]
            response = response.strip()

            print(f"Attempt {attempt + 1} Response: {response}")

            parsed_response = json.loads(response)
            return parsed_response

        except json.JSONDecodeError as e:
            print(f"JSON parsing error on attempt {attempt + 1}: {e}")
            if attempt == max_retries - 1:
                return {str(i+1): {"unfairly_downvoted": None, "rule_violated": None}
                        for i in range(len(comments))}

        except Exception as e:
            print(f"Other error on attempt {attempt + 1}: {e}")
            if attempt == max_retries - 1:
                return {str(i+1): {"unfairly_downvoted": None, "rule_violated": None}
                        for i in range(len(comments))}

        time.sleep(2)

In [ ]:
from openai import OpenAI
import httpx
client = OpenAI(
    api_key="glhf_af1fda11547e49d9eebf4c67b7ea60c7",
    base_url="https://glhf.chat/api/openai/v1",
    http_client=httpx.Client(verify=False)
)

def process_subreddit(subreddit_info):
    print(f"\nProcessing {subreddit_info['subreddit']}...")

    df = pd.read_csv(subreddit_info['csv_path'])
    total_comments = len(df)

    unfairly_downvoted_results = []
    rules_violated = []

    chunk_size = 20
    for i in range(0, total_comments, chunk_size):
        chunk = df['Comment'].iloc[i:i + chunk_size]
        try:
            print(f"\nProcessing Batch {i//chunk_size + 1}/{(total_comments + chunk_size - 1)//chunk_size}")
            results = analyze_comments_batch(chunk.tolist(), subreddit_info['rules'])

            for j in range(len(chunk)):
                if str(j + 1) in results:
                    unfairly_downvoted_results.append(results[str(j + 1)]['unfairly_downvoted'])
                    rules_violated.append(results[str(j + 1)]['rule_violated'])
                else:
                    unfairly_downvoted_results.append(None)
                    rules_violated.append(None)
        except Exception as e:
            print(f"\nError processing batch starting at index {i}")
            print(f"Error: {e}")
            for _ in range(len(chunk)):
                unfairly_downvoted_results.append(None)
                rules_violated.append(None)

    unfairly_downvoted_results = unfairly_downvoted_results[:total_comments]
    rules_violated = rules_violated[:total_comments]

    while len(unfairly_downvoted_results) < total_comments:
        unfairly_downvoted_results.append(None)
    while len(rules_violated) < total_comments:
        rules_violated.append(None)

    df['LLM_unfairly_downvoted'] = unfairly_downvoted_results
    df['LLM_rule_violated'] = rules_violated

    output_file = f'{subreddit_info["subreddit"][2:]}_comments_with_analysis.csv'
    df.to_csv(output_file, index=False)

    return df

In [ ]:
results = {}
for subreddit_info in subreddit_data:
    results[subreddit_info['subreddit']] = process_subreddit(subreddit_info)
    print(f"\nCompleted processing {subreddit_info['subreddit']}")
    print(f"Results saved to {subreddit_info['subreddit'][2:]}_comments_with_analysis.csv")

for subreddit, df in results.items():
    print(f"\nResults for {subreddit}:")
    print(df[['Comment', 'LLM_unfairly_downvoted', 'LLM_rule_violated']])

Streaming output truncated to the last 5000 lines.
        "rule_violated": "3b. Flame bait"
    },
    "9": {
        "unfairly_downvoted": false,
        "rule_violated": "3b. Flame bait"
    },
    "10": {
        "unfairly_downvoted": false,
        "rule_violated": "3b. Flame bait"
    },
    "11": {
        "unfairly_downvoted": true,
        "rule_violated": "null"
    },
    "12": {
        "unfairly_downvoted": false,
        "rule_violated": "3b. Flame bait"
    },
    "13": {
        "unfairly_downvoted": true,
        "rule_violated": "null"
    },
    "14": {
        "unfairly_downvoted": true,
        "rule_violated": "null"
    },
    "15": {
        "unfairly_downvoted": false,
        "rule_violated": "3b. Flame bait"
    },
    "16": {
        "unfairly_downvoted": false,
        "rule_violated": "3b. Flame bait"
    },
    "17": {
        "unfairly_downvoted": false,
        "rule_violated": "3b. Flame bait"
    },
    "18": {
        "unfairly_downvoted": false,
   